In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
train_summaries = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
train_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

In [3]:
train_data = pd.merge(train_summaries, train_prompts, on="prompt_id")

In [4]:
X = train_data["text"]
y_content = train_data["content"]
y_wording = train_data["wording"]

In [5]:
X_train, X_valid, y_content_train, y_content_valid, y_wording_train, y_wording_valid = train_test_split(
    X, y_content, y_wording, test_size=0.2, random_state=42
)

In [6]:
max_words = 10000  
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_valid_tokens = tokenizer.texts_to_sequences(X_valid)

In [7]:
max_sequence_length = 500  # Adjust based on your data
X_train_padded = pad_sequences(X_train_tokens, maxlen=max_sequence_length, padding='post')
X_valid_padded = pad_sequences(X_valid_tokens, maxlen=max_sequence_length, padding='post')

In [8]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))  # Regression task

model.compile(loss='mean_squared_error', optimizer='adam')

In [9]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

In [10]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [11]:
model.fit(X_train_padded, y_content_train, validation_data=(X_valid_padded, y_content_valid), epochs=10, batch_size=32)

Epoch 1/10
180/180 [==============================] - 176s 946ms/step - loss: 1.0936 - val_loss: 1.0840
Epoch 2/10
180/180 [==============================] - 173s 961ms/step - loss: 1.0919 - val_loss: 1.0829
Epoch 3/10
180/180 [==============================] - 177s 983ms/step - loss: 1.0907 - val_loss: 1.0829
Epoch 4/10
180/180 [==============================] - 178s 991ms/step - loss: 1.0900 - val_loss: 1.0829
Epoch 5/10
180/180 [==============================] - 177s 982ms/step - loss: 1.1040 - val_loss: 1.0829
Epoch 6/10
180/180 [==============================] - 174s 969ms/step - loss: 1.0874 - val_loss: 1.0831
Epoch 7/10
180/180 [==============================] - 172s 954ms/step - loss: 1.0893 - val_loss: 1.0829
Epoch 8/10
180/180 [==============================] - 170s 944ms/step - loss: 1.0851 - val_loss: 1.0829
Epoch 9/10
180/180 [==============================] - 168s 935ms/step - loss: 1.0841 - val_loss: 1.0829
Epoch 10/10
180/180 [==============================] - 169s 940m

In [12]:
test_summaries = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv")
test_prompts = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv")

test_data = pd.merge(test_summaries, test_prompts, on="prompt_id")
X_test = test_data["text"]

X_test_tokens = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_tokens, maxlen=max_sequence_length, padding='post')

test_predictions_content = model.predict(X_test_padded)
test_predictions_wording = model.predict(X_test_padded) 

1/1 [==============================] - 0s 105ms/step


In [13]:
submission = pd.DataFrame({
    "prompt_id": test_data["prompt_id"],
    "content": test_predictions_content.flatten(),
    "wording": test_predictions_wording.flatten()
})

submission.to_csv("submission.csv", index=False)